Test scrapping on a few specified pages.

In [1]:
import requests
from bs4 import BeautifulSoup
from bs4 import Comment

test cases:

In [2]:
domain = 'https://www.kochbar.de'
urls = ['/rezept/443097/Abendbrot-UEberbackenes-Broetchen-a-la-Gudrun.html','/rezept/108268/Coq-au-vin.html','/rezept/231469/Kuchen-Blechkuchen-Kaesekuchen-mit-Mohn-und-Eierlikoer.html']
results = [['Auflauf /Überbackenes','Fleisch','Milchprodukte','Frühstück','Abendessen','Date','Party','preiswerte',
 'schnell & einfach','glutenfrei','ohne Weizen'], ['Europa', 'Frankreich', 'Fleisch', 'Hauptspeise', 'ohne Weizen'], 
          ['Kuchen/Torte', 'Kaffee & Kuchen', 'glutenfrei', 'ohne Weizen']]

In [3]:
response = requests.get(domain + urls[1])
soup = BeautifulSoup(response.content.decode('utf-8','ignore'), "lxml")

check response status

In [4]:
response.status_code

200

check chars encoding

In [5]:
comments = soup.find_all(string=lambda text:isinstance(text, Comment))
comments[89]

' ÄHNLICHE KATEGORIEN '

## Find first node where to start looking for siblings

In [6]:
def find_comment(soup, wanted_comment='REZEPT-KATEGORIEN'):
    found = False

    comments = soup.find_all(string=lambda text:isinstance(text, Comment))
    i = 0
    while not found and i < len(comments):
        node = comments[i]
        if node.strip() == wanted_comment:
            #print('comment found')
            found = True

        i+=1
    return node

find_comment(soup)

' REZEPT-KATEGORIEN '

## Find categories

In [7]:
def find_categories(soup, wanted_class = 'rtli-pb-small'):
    data = []
    node = find_comment(soup)
    startOfItems, endOfItems = False, False
    
    while not endOfItems and node.find_next_sibling():
        node = node.find_next_sibling()

        if node['class'][0] == wanted_class:
            startOfItems = True

            if node.findChild().name == 'a':
                data.append(node.a.text)
            elif node.findChild().name == 'span':
                data.append(node.span.text)

        elif startOfItems:
            endOfItems = True
    
    return data
                
find_categories(soup)

['Europa', 'Frankreich', 'Fleisch', 'Hauptspeise', 'ohne Weizen']

# Launch tests

In [9]:
for i in range(len(urls)):
    response = requests.get(domain + urls[i])
    
    if response.url != 'https://www.kochbar.de/rezepte/' and response.status_code == 200:
        print('Request status ok!')
        soup = BeautifulSoup(response.content.decode('utf-8','ignore'), "lxml")
        categories = find_categories(soup)
        print('\tcategories:', categories, '\n')
    elif response.status_code == 200:
        print('This webpage is probably no longer available!')
    else:
        print('Not correct request status: ', response.status_code)

Request status ok!
	categories: ['Auflauf /Überbackenes', 'Fleisch', 'Milchprodukte', 'Frühstück', 'Abendessen', 'Date', 'Party', 'preiswerte', 'schnell & einfach', 'glutenfrei', 'ohne Weizen'] 

Request status ok!
	categories: ['Europa', 'Frankreich', 'Fleisch', 'Hauptspeise', 'ohne Weizen'] 

Request status ok!
	categories: ['Kuchen/Torte', 'Kaffee & Kuchen', 'glutenfrei', 'ohne Weizen'] 

